# Imports 

In [ ]:
# All imports needed for this research, and mount to the google drive where the data was stored
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from music21 import *
import re
from google.colab import files
import os
import music21
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.tree import export_text
from sklearn.datasets import load_iris
from sklearn import metrics
drive.mount('/content/drive')

# Import midi files from google drive 
  

In [ ]:
# All MIDI files were stored on my google drive, therefore some functions need to be applied to extract these from google drive and convert them to midi21 objects
# ------------------------------------------- 
base_midi = []
survey_songs = []

# Google drive links
survey_song = os.listdir('/content/drive/MyDrive/Music and AI: Composing music as a logical modeling problem/Surveysongsmidifiles/')
classical_songs = os.listdir('/content/drive/MyDrive/Music and AI: Composing music as a logical modeling problem/Composersmidifiles/')
pop_songs = os.listdir('/content/drive/MyDrive/Music and AI: Composing music as a logical modeling problem/Popsongsmidifiles/')

# ------------------------------------------- 
#Functions to extract songs from google drive and convert them to music21 Objects
def extract_survey_songs(song):
  string = '/content/drive/MyDrive/Music and AI: Composing music as a logical modeling problem/Surveysongsmidifiles/' + song
  survey_songs.append((converter.parse(string), song))

def extract_classical_songs(composer):
  string = '/content/drive/MyDrive/Music and AI: Composing music as a logical modeling problem/Composersmidifiles/' + composer + '/'
  folder = os.listdir(string)
  for song in folder:
    string_song = string + song
    base_midi.append((converter.parse(string_song), song))

def extract_pop_songs(song):
  string = '/content/drive/MyDrive/Music and AI: Composing music as a logical modeling problem/Popsongsmidifiles/' + song
  base_midi.append((converter.parse(string), song))

# -------------------------------------------
#For loops to extract songs from different google drive links
for i in survey_song:
  extract_survey_songs(i)
  
for i in classical_songs:
  extract_classical_songs(i)

for i in pop_songs:
  extract_pop_songs(i)

# Feature functions

In [88]:
# Function to analyze the key of a song 
def analyze_key(song):
  print(song)
  key = str(song[0].analyze('key'))
  return key

# Function to extraxt the key confidence of a song 
def music_key_confidence(song):
  music_analysis = song[0].analyze('key')
  key_confidence = music_analysis.correlationCoefficient
  return key_confidence
 
# Function to extract the minimal pitch of a song 
def min_pitch(song):
  notes = song[0].flat.getElementsByClass(music21.note.Note)
  min_pitch = min([note.pitch.midi for note in notes])
  return(min_pitch)

# Function to extract the maximal pitch of a song 
def max_pitch(song):
  notes = song[0].flat.getElementsByClass(music21.note.Note)
  max_pitch = max([note.pitch.midi for note in notes])
  return max_pitch

# Function to extract the pitch range of a song
def pitch_range(song):
  notes = song[0].flat.getElementsByClass(music21.note.Note)
  min_pitch = min([note.pitch.midi for note in notes])
  max_pitch = max([note.pitch.midi for note in notes])
  pitch_range = max_pitch - min_pitch 
  return pitch_range

# Function to extract the velocity of a song
def velocity(song):
  notes = song[0].flat.getElementsByClass(music21.note.Note)
  mean_velocity = sum([note.volume.velocity for note in notes])/len(notes)
  return mean_velocity

# Function to extraxt the time signature of a song 
def time_signature(song):
  timeSignature = song[0].getTimeSignatures()[0] 
  name_of_song = song[1]
  time_signature = "{0}/{1}".format(timeSignature.beatCount, timeSignature.denominator)
  return time_signature

# Funtcion to extract the bpm of a song 
def find_bpm(song):
  tempo = song[0].flat.getElementsByClass(music21.tempo.MetronomeMark)[0]
  bpm = tempo.number
  return bpm

# Survey dataframe


In [ ]:
features = []

# For loop to extract feature's from the musical pieces used within the survey
for song in survey_songs:
  key = analyze_key(song)
  key_confidence = music_key_confidence(song)
  minimal_pitch  = min_pitch(song)
  maximal_pitch = max_pitch(song)
  range_of_pitch = pitch_range(song)
  rhythm  = time_signature(song)
  bpm = int(find_bpm(song))
  vel = int(velocity(song))
  features.append((key, key_confidence, minimal_pitch, maximal_pitch, range_of_pitch, rhythm, bpm, vel, song[1]))

# Creating the Panda Dataframe for the musical pieces used in the survey
features_survey_dataframe = pd.DataFrame(features, columns = ['key', 'key confidence', 'min pitch', 'max pitch', 'pitch_range', 'time signature','bpm', 'velocity', 'name'])
features_survey_dataframe.set_index('name', inplace = True)
boolean_survey_dataframe = pd.get_dummies(data=features_survey_dataframe , columns=['key', 'time signature'])
display(boolean_survey_dataframe )


# Survey 



In [ ]:
# Import data collected by the survey
uploaded = files.upload()

In [91]:
# Reed the data collected by the survey
df = pd.read_csv('survey.csv')

In [92]:
# Rename all the columns to the corresponding name of the song
df.rename(columns = {'Horodateur': 'Timestamp',
                     'Which color do you associate with Track 1': 'The Corrs - Dreams',
                     'Which color do you associate with Track 2?': 'Mozart - 333 3',
                     'Which color do you associate with Track 3?': 'Beethoven - opus22 4',
                     'Which color do you associate with Track 4?': 'Chopin  - op18', 
                     'Which color do you associate with Track 5?': 'Frank Mills - Musicbox Dancer',
                     'Which color do you associate with Track 6?': 'Borodin - ps7',
                     'Which color do you associate with Track 7?': 'Cyndi Lauper - True Colors',
                     'Which color do you associate with Track 8?': 'George Benson - On Broadway',
                     'Which color do you associate with Track 9?': 'Grieg - Voeglein',
                     'Which color do you associate with Track 10?': 'ABBA - Money Money Money',
                     'How old are you': "Age",
                     'What is your gender': 'Gender',}, inplace = True)

In [93]:
# Drop the columns not relevant for this research
df = df.drop(columns = ['Timestamp', 'Age', 'Gender'])

In [94]:
# Returns how many time participant did vote for a certain color
all_genders_colors = df.copy()
all_genders_colors = all_genders_colors.apply(pd.value_counts)
all_genders_colors = all_genders_colors.fillna(0)

In [ ]:
# Create pie charts for each song used within the survey
for column in all_genders_colors:
    fig, (ax1) = plt.subplots(1, 1,  figsize=(18,6))

    all_genders_colors.plot.pie( 
                            y = all_genders_colors[column].name,
                            colors = ['#625f64', '#6fa8dc', '#8fce00','Pink', '#b892dc' , '#e75454', 'White', '#ffd966'],
                            legend = False,
                            subplots = True,
                            label = '',
                            wedgeprops = {"edgecolor":"k", "linewidth": 0.5, "linestyle": "solid", "antialiased": True},
                            autopct=lambda p: '{:.1f}%'.format(round(p)) if p > 0 else "",
                            ax = ax1)
  
    ax1.set_title(column)
    plt.tight_layout()
    nameFigure = "{}_Gender_color.png".format(column)
    plt.savefig(nameFigure)


In [96]:
# Creates a dataframe with the color that is most present of each song
color_each_song = []

for column in all_genders_colors:
  name  = column + '.mid'
  color_each_song.append((all_genders_colors[column].idxmax(), name))

color_each_song

df_color = pd.DataFrame(color_each_song, columns = ['Color', 'name'])
df_color.set_index('name', inplace=True)

#  Dataframe survey with color of each song


In [ ]:
# Merges the color dataframe and feature dataframe of the survey together, and applies one-hot-encoding to some features of this dataframe
features_survey_dataframe_with_color = pd.merge(features_survey_dataframe, df_color, on='name')
pd.get_dummies(features_survey_dataframe_with_color, columns=['key', 'time signature'])

# Extracting feature's from each color set

In [ ]:
# To extract all feature's from each color set, the color within each link needs to be changed to the specific color
google_drive_link = '/content/drive/MyDrive/Music and AI: Composing music as a logical modeling problem/Greenmidifiles/' 
music_google_drive = os.listdir('/content/drive/MyDrive/Music and AI: Composing music as a logical modeling problem/Greenmidifiles/')
all_music = []
color = 'Green'
Name_to_save_data = 'Green'

# Function to extract from google drive
def extract_songs(song):
  string = '/content/drive/MyDrive/Music and AI: Composing music as a logical modeling problem/Greenmidifiles/' + song
  all_music.append([converter.parse(string), song])
 
# For loops to extract songs from google drive links
for song in music_google_drive:
  extract_songs(song)

In [ ]:
features_2 = []

# For loop to extract feature's from the musical pieces within each color
for song in all_music:
  key = analyze_key(song)
  key_confidence = music_key_confidence(song)
  minimal_pitch  = min_pitch(song)
  maximal_pitch = max_pitch(song)
  range_of_pitch = pitch_range(song)
  rhythm  = time_signature(song)
  bpm = int(find_bpm(song))
  vel = int(velocity(song))
  features_2.append((key, key_confidence, minimal_pitch, maximal_pitch, range_of_pitch, rhythm, bpm , vel, song[1]))


# Dataframe
features_dataframe_2 = pd.DataFrame(features_2, columns = ['key', 'key confidence', 'min pitch', 'max pitch', 'pitch_range', 'time signature', 'bpm', 'velocity', 'name'])
features_dataframe_2.set_index('name', inplace = True)


In [ ]:
features_dataframe_2.insert(loc = len(features_dataframe_2.columns), column = 'Color', value = color)

In [ ]:
boolean_dataframe = pd.get_dummies(data=features_dataframe_2, columns=['key', 'time signature'])
display(boolean_dataframe)
csv_data = boolean_dataframe.to_csv(Name_to_save_data)

# Import and merge data





In [121]:
# Import dataframes from every color
uploaded = files.upload()

In [120]:
# Save all data as panda dataframes
df_black = pd.read_csv('Black.csv')
df_blue = pd.read_csv('Blue.csv')
df_green = pd.read_csv('Green.csv')
df_pink = pd.read_csv('Pink.csv')
df_purple = pd.read_csv('Purple.csv')
df_red = pd.read_csv('Red.csv')
df_white = pd.read_csv('White.csv')
df_yellow = pd.read_csv('Yellow.csv')


In [119]:
# Merging all the different color dataframes into one big dataframe and fill all the NaN with 0's
final_dataframe = pd.concat([df_black, df_blue, df_green, df_pink, df_purple, df_red, df_white, df_yellow], ignore_index=True)
dataframe_final = final_dataframe.fillna(0)

In [118]:
# Merging all the blue and black dataframe together and fill all the NaN with 0's
df_blue_black = pd.concat([df_blue, df_black], ignore_index=True)
dataframe_blue_black = df_blue_black.fillna(0)

In [117]:
# Merging all the Yellow and Pruple dataframe together and fill all the NaN with 0's
df_yellow_purple = pd.concat([df_yellow, df_purple], ignore_index=True)
dataframe_yellow_purple = df_yellow_purple.fillna(0)

# Decision tree


In [116]:
# Since not all features in the final dataframe are present within the datframe of the survey, it is needed for the decision classifier the add these features to the dataframe of the survey
cols = features_survey_dataframe_with_color.columns.union(dataframe_final.columns)
df_survey = features_survey_dataframe_with_color.reindex(cols, axis=1, fill_value = 0)

In [ ]:
#load Data
dataframe = dataframe_final

# Separate Target Variable and Predictor Variables
# Creating a list with predicor labels 
columns_names = dataframe.columns.values.tolist()
columns_names.remove('Color')
columns_names.remove('name')
columns_names.remove('key confidence')

# Prediction and Target values 
Predictors = columns_names
TargetVariable = ['Color']
X = dataframe[Predictors].values
y = dataframe[TargetVariable].values

# Test data of the survey
X_2 = df_survey[Predictors].values
y_2 = df_survey[TargetVariable].values

# Split the data intro training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

# Choose from different tunable hyper paramenters
clf = tree.DecisionTreeClassifier(max_depth = 5, criterion = 'entropy', splitter = 'best',)

# Creating the model on training Data
DTree = clf.fit(X_train, y_train)
prediction = DTree.predict(X_test)

# Measuring accuracy on testing data
print(metrics.classification_report(y_test, prediction))

In [ ]:
# Measuring accuracy on testing data
prediction = DTree.predict(X_2)
print(metrics.classification_report(y_2, prediction))

In [ ]:
# Text visualisation 
text_visualisation = export_text(DTree, feature_names=Predictors)
print(r)

In [ ]:
# All target values for the different datasets
targets_all_data = ['Black', 'Blue', 'Green', 'Pink', 'Purple', 'Red', 'White', 'Yellow']
targets_purple_yellow = ['Purple', 'Yellow']
targets_blue_black = ['Black', 'Blue']

#Decision tree visualisation
plt.figure(figsize=(25,25))
tree.plot_tree(DTree, feature_names = Predictors, class_names = targets_all_data, label ='all', impurity = False, fontsize=3)
plt.savefig('Decisiontree_all_data.jpg',format='jpg', bbox_inches = "tight", dpi=900)